In [1]:
!pip install transformers argostranslate textsum optimum onnxruntime-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.6/380.6 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.1 MB/s eta 0:00

In [11]:
!pip install optimum[onnxruntime]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.5 MB/s eta 0:00:00


In [19]:
import duckdb
import os
import numpy as np
import torch
import argostranslate.package
import argostranslate.translate
from multiprocessing import Process, cpu_count
from textsum.summarize import Summarizer
import gc

In [25]:
gc.collect()

90

In [3]:
num_cores = cpu_count()
print(f"Quantidade de núcleos: {num_cores}")

Quantidade de núcleos: 2


In [4]:
if torch.cuda.is_available():
    device = torch.cuda.get_device_name(0)
    print("GPU encontrada:", device)
else:
    print("Nenhuma GPU disponível.")

GPU encontrada: Tesla T4


In [6]:
os.environ["ARGOS_DEVICE_TYPE"]="cuda"

In [30]:
# Conectar (ou criar) um banco de dados
con = duckdb.connect(database='jus.duckdb', read_only=False)

In [ ]:
con.execute("""
CREATE TABLE metadadosPublicacao202202 AS SELECT * FROM read_json_auto('./dataset/metadadosPublicacao202202.json');
""")

In [ ]:
# tabela de texto dos juiz
con.execute("""
  CREATE TABLE decisions_description (
    ID INT PRIMARY KEY,
    content TEXT,
    summarize TEXT
  );
""")

In [ ]:
# Download and install Argos Translate package
argostranslate.package.update_package_index()
available_packages = argostranslate.package.get_available_packages()
package_to_install = next(
    filter(
        lambda x: x.from_code == "pt" and x.to_code == "en", available_packages
    )
)
argostranslate.package.install_from_path(package_to_install.download())

# Translate
translatedText = argostranslate.translate.translate("Quais são os benefícios de banana ?", "pt", "en")
print(translatedText)

What are the benefits of banana?


In [ ]:
con.execute("""
DELETE FROM metadadosPublicacao202202 WHERE teor NOT IN ('Concedendo', 'Negando') or teor IS NULL;
""")

In [ ]:
concedendo_label = 1
negando_label = 0

In [ ]:
data = con.execute("SELECT seqDocumento FROM metadadosPublicacao202202").fetchnumpy()
data["seqDocumento"]

array([144948780, 145620297, 139579332, ..., 142924179, 145440916,
       144826578])

In [ ]:
for subdir, dirs, files in os.walk('./dataset/'):
    for filename in files:
        if filename.endswith('.txt'):
            file_path = os.path.join(subdir, filename)
            # Extrai o ID do nome do arquivo
            file_id = int(filename.split('.')[0])
            if np.where(data["seqDocumento"] == file_id)[0].size == 1:
                with open(file_path, 'r') as f:
                    content = f.read()
                    translatedText = argostranslate.translate.translate(content, "pt", "en")
                    try:
                        con.execute('INSERT INTO decisions_description (id,content) VALUES (?,?)', (file_id, translatedText))
                    except Exception as e:
                        if 'violates primary key constraint' in str(e):
                            print("Duplicate key, ignoring.")
                        else:
                            raise
                    print(index, file_id)

Duplicate key, ignoring.
3 144943042
Duplicate key, ignoring.
4 145612097
Duplicate key, ignoring.
5 145620627
Duplicate key, ignoring.
6 145480544


In [14]:
summarizer = Summarizer(
    model_name_or_path="pszemraj/long-t5-tglobal-base-16384-book-summary",
    num_beams=4,
    force_cache=True,
    #optimum_onnx=True,
    compile_model=True
)

In [46]:
data = con.execute("SELECT ID, content FROM decisions_description").fetchnumpy()
data

{'ID': array([121184492, 122027196, 123312824, ..., 143346074, 143349037,
        143349119], dtype=int32),
 'content': array(['DECISION > It is treated of habeas corpus impetrado for the benefit of Valter de Oliveira Albano - fulfilling penalty in the Center for Resocialization of Lins/SP, currently in closed regime, with the end of the sentence provided for 19/3/2027 -, pointing itself as coactor body maintaining the Court of Justice of São Paulo, which ordered the HC no. 21346-19-77.20 The judgment has this issue (f. 82):<br>HABEAS CORPUS - CRIMINAL EXECUTION - Impetration aiming at granting home imprisonment by integrating risk group, carrier of hypertension - Impossibility - Absence of cabal proof that the prison unit that houses it does not have adequate structure to keep it, nor to offer it,47 if necessary, the appropriate treatment Recommendation No 62 of the CNJ which has no binding character - Implementation of measures To contain the spread of the disease in prison units - S

In [ ]:
for index, element in enumerate(data["content"]):
  if index >= 3 and index <= 60:
    out_str = summarizer.summarize_string(element)
    print(index,data["ID"][index],len(out_str))

    con.execute("""
      UPDATE decisions_description
      SET summarize = $out_str
      WHERE ID = $id;
      """,{
        'out_str': out_str,
        'id': int(data["ID"][index]),
      })

con.commit()

Generating Summaries:   0%|          | 0/2 [00:00<?, ?it/s]

3 123447928 1093


Generating Summaries:   0%|          | 0/4 [00:00<?, ?it/s]

4 123917739 2304


Generating Summaries:   0%|          | 0/3 [00:00<?, ?it/s]

...

Generating Summaries:   0%|          | 0/1 [00:00<?, ?it/s]

6 125636167 648


Generating Summaries:   0%|          | 0/3 [00:00<?, ?it/s]

7 127181699 2916


Generating Summaries:   0%|          | 0/10 [00:00<?, ?it/s]

In [60]:
con.commit()
con.close()

In [62]:
con = duckdb.connect(database='jus.duckdb', read_only=False)